### Transforming the GK DIMTable 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark= SparkSession.builder.getOrCreate()

In [3]:
GoalKeepers = spark.read.format('csv')\
    .option("header", "True")\
    .option("inferSchema", "True")\
    .load("../data/goalkeepers.csv")

In [4]:
GoalKeepers.columns

['LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'SoTA',
 'GA',
 'Saves',
 'PSxG',
 'PKatt',
 'PKA',
 'PKm',
 'PassAtt',
 'Throws',
 'AvgLen',
 'GKAtt',
 'GKAvgLen',
 'C']

In [5]:
GoalKeepers.createOrReplaceTempView("GK")

In [6]:
GoalKeepers= GoalKeepers.withColumn('fullname', concat_ws(" ", col('FirstName'),col('LastName')))
GoalKeepers.select("fullname").show(5, False)

+------------+
|fullname    |
+------------+
|David Ospina|
|Petr Cech   |
|Petr Cech   |
|David Ospina|
|David Ospina|
+------------+
only showing top 5 rows



In [7]:
GoalKeepers_f = spark.sql("""
    select distinct concat(firstname, " ", lastname) as fullname
    from GK

""")


In [8]:
GoalKeepers_f.columns

['fullname']

In [10]:
GK_ = spark.sql ("""
select Count(distinct full_name) from GK

""").show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `full_name` cannot be resolved. Did you mean one of the following? [`LastName`, `Date`, `FirstName`, `Min`, `PKm`].; line 2 pos 22;
'Project [unresolvedalias('Count(distinct 'full_name), None)]
+- SubqueryAlias gk
   +- View (`GK`, [LastName#17,FirstName#18,Date#19,Start#20,Pos#21,Min#22,SoTA#23,GA#24,Saves#25,PSxG#26,PKatt#27,PKA#28,PKm#29,PassAtt#30,Throws#31,AvgLen#32,GKAtt#33,GKAvgLen#34,C#35])
      +- Relation [LastName#17,FirstName#18,Date#19,Start#20,Pos#21,Min#22,SoTA#23,GA#24,Saves#25,PSxG#26,PKatt#27,PKA#28,PKm#29,PassAtt#30,Throws#31,AvgLen#32,GKAtt#33,GKAvgLen#34,C#35] csv


In [ ]:
GoalKeepers_f= GoalKeepers_f.withColumn('GkID',monotonically_increasing_id()+1)


In [ ]:
GoalKeepers_f.show()

In [ ]:
DimGoalKeepers= GoalKeepers.join(GoalKeepers_f, on ='fullname', how="inner")

In [29]:
DimGoalKeepers.show()

+------------+--------+---------+----------+-----+---+---+----+---+-----+----+-----+----+----+-------+------+------+-----+--------+---+----+
|    fullname|LastName|FirstName|      Date|Start|Pos|Min|SoTA| GA|Saves|PSxG|PKatt| PKA| PKm|PassAtt|Throws|AvgLen|GKAtt|GKAvgLen|  C|GkID|
+------------+--------+---------+----------+-----+---+---+----+---+-----+----+-----+----+----+-------+------+------+-----+--------+---+----+
|David Ospina|  Ospina|    David| 5/13/2018|    1| GK| 90|   3|  0|    3| 0.4|    0|   0|   0|     39|     8|  31.9|    9|    48.2|  0|   7|
|   Petr Cech|    Cech|     Petr|  5/9/2018|    1| GK| 90|  10|  3|    7| 3.2|    1|   1|   0|     26|     7|  34.5|   11|    66.0|  1|   4|
|   Petr Cech|    Cech|     Petr|  5/6/2018|    1| GK| 90|   2|  0|    2| 0.2|    0|   0|   0|     31|     8|  32.4|    2|    56.0|  1|   4|
|David Ospina|  Ospina|    David| 4/29/2018|    1| GK| 90|   2|  2|    0| 1.3|    0|   0|   0|     15|     4|  41.1|    5|    49.0|  0|   7|
|David Ospina

In [31]:
DimGoalKeepers.columns

['fullname',
 'LastName',
 'FirstName',
 'Date',
 'Start',
 'Pos',
 'Min',
 'SoTA',
 'GA',
 'Saves',
 'PSxG',
 'PKatt',
 'PKA',
 'PKm',
 'PassAtt',
 'Throws',
 'AvgLen',
 'GKAtt',
 'GKAvgLen',
 'C',
 'GkID']

In [32]:
DimGoalKeepers.write.csv('../data/DimGoalkaeeper', header=True)

### Creating the FactGK 

In [35]:
DimMatches = spark.read.format('csv')\
    .option("header", "True")\
    .option("inferSchema", "True")\
    .load("../data/DimMatches/DimMatches.csv")

In [11]:
FactGk = DimMatches.join(DimGoalKeepers, on='Date', how='left')
FactGk.columns

NameError: name 'DimMatches' is not defined

In [42]:
FactGK = FactGk.drop( 
 'Season',
 'Tour',
 'Time',
 'Opponent',
 'HoAw',
 'Stadium',
 'Coach',
 'Referee',
  'Pos',
    'C',
'fullname',
 'LastName',
 'FirstName'
 
)

In [43]:
FactGK.columns

['Date',
 'ArsenalScore',
 'OpponentScore',
 'Attendance',
 'MatchID',
 'Start',
 'Min',
 'SoTA',
 'GA',
 'Saves',
 'PSxG',
 'PKatt',
 'PKA',
 'PKm',
 'PassAtt',
 'Throws',
 'AvgLen',
 'GKAtt',
 'GKAvgLen',
 'GkID']

In [44]:
FactGK.write.csv('../data/FactGK', header=True)